## Import All Required Libraries

In [1]:
# Import all connection libraries

from spotipy import SpotifyClientCredentials
import spotipy
import sqlalchemy as sql
import psycopg2

# Import all data manipulation libraries

import pandas as pd
from pandarallel import pandarallel
import numpy as np
from tqdm import tqdm
pandarallel.initialize(verbose=0, nb_workers=8)

In [2]:
from functools import lru_cache

# Use the lru_cache decorator to cache the result of the function
# maxsize=1000 means the cache will store the result of up to 1000 items
@lru_cache(maxsize=1000)
def get_playlist_tracks(playlist_uri):
    """
    This function uses the Spotify API to get the tracks of a given playlist.
    The function is decorated with the lru_cache decorator to cache the result for each unique playlist_uri.
    So, if the same playlist_uri is passed to the function again, the cached result will be returned
    instead of making a new API call.
    :param playlist_uri: The Spotify URI of the playlist
    :return: A DataFrame containing the tracks of the playlist
    """
    # Use the Spotify API to get the tracks of the playlist
    tracks = pd.json_normalize(sp.playlist_tracks(playlist_uri), record_path=['items'])
    return tracks

## Load Initial Training Data

In [3]:
# establish connection to Spotify API

cid = 'b1d63d0b2fcc4b379cf7b7ad0d9d9fc0'
secret = '87f181a928e04a40977a281e9900e3b9'
client_credentials = SpotifyClientCredentials(client_id=cid, client_secret=secret)
sp = spotipy.Spotify(client_credentials_manager=client_credentials, requests_timeout=15, retries=10)

# establish connection to Postgres

host_name = 'localhost'
database_name = 'Spotify'
engine = sql.create_engine('postgresql+psycopg2://postgres:DataNerd2023!!\
@localhost/Spotify')

# load training data
new_batch = pd.read_csv('distinct_playlists.csv')[['playlist_uri', 'playlist_name']]
new_batch['playlist_uri'] = new_batch['playlist_uri'].str.strip()

db_query = pd.read_sql('SELECT DISTINCT playlist_uri FROM playlist_tracks', engine)

outer = new_batch.merge(db_query, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

new_batch = pd.DataFrame(anti_join)
new_batch = new_batch[0:25]
new_batch.head()

,playlist_uri,playlist_name
21,spotify:playlist:01clVI1RPrF8zHMxrqVIs0,2020 top rap and hiphop hits 2022
34,spotify:playlist:02eILNCPJV0hoYxMUreT0L,He is back
114,spotify:playlist:07Kw4TRKVUaIzJdiA86Kvu,slow down house
144,spotify:playlist:0ABVkZSG9cLTOLl52r8ha6,Albanian 2022 | Top 100
145,spotify:playlist:0ACKSvSP5MsCdLpUuxRPwd,Today's Top Hits


## Iterate Through Training Data to Change Grain of Data

In [4]:
load_batch = []
series = new_batch['playlist_uri'].to_dict()
for playlist in tqdm(series.values()):
    try:
        tracks = get_playlist_tracks(playlist)
        if len(tracks) >= 100:
                tracks2 = pd.json_normalize(sp.playlist_tracks(playlist, offset=100), record_path=['items'])
                tracks3 = pd.json_normalize(sp.playlist_tracks(playlist, offset=200), record_path=['items'])
                tracks = pd.concat([tracks, tracks2, tracks3])
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])

                load_batch.append(tracks)
        else:
                secondary_data = pd.json_normalize(sp.playlist(playlist))
                tracks['playlist_uri'] = playlist
                tracks = tracks.merge(secondary_data, left_on='playlist_uri', right_on='uri')
                tracks = tracks.rename(columns={"name":"playlist_name", "track.name":"track_name", "track.uri":"track_uri", "track.album.name":"album_name", "track.explicit":"isExplicit", "track.album.release_date":"release_date", "track.duration_ms":"duration_ms", "track.album.uri":"album_uri", "added_by.external_urls.spotify": "added_by_external_urls_spotify", 'added_by.href':"added_by_href", "added_by.id":"added_by_id", "added_by.type":"added_by_type", "added_by.uri":"added_by_uri", "track.album.album_type":"track_album_album_type", "track.album.external_urls.spotify":"track_album_external_urls_spotify", "track.album.href":"track_album_href", "track.album.id":"track_album_id", "track.album.release_date_precision":"track_album_release_date_precision", "track.album.total_tracks":"track_album_total_tracks","track.album.type":"track_album_type", "track.disc_number":"track_disc_number", "track.episode":"track_episode", "track.external_ids.isrc":"track_external_ids_isrc", "track.external_urls.spotify":"track_external_ids_spotify", "track.href":"track_href", "track.id":"track_id", "track.is_local":"track_is_local", "track.popularity":"track_popularity", "track.preview_url":"track_preview_url", "track.track":"track_track", "track.track_number":"track_track_number", "track.type":"track_type", "video_thumbnail.url":"video_thumbnail_url", "external_urls.spotify":"external_urls_spotify", "followers.href":"followers_href", "followers.total":"followers_total", "owner.display_name":"owner_display_name", "owner.external_urls.spotify":"owner_external_urls_spotify", "owner.href":"owner_href", "owner.id":"owner_id", "owner.type":"owner_type", "owner.uri":"owner_uri", "tracks.href":"tracks_href", "tracks.limit":"tracks_limit", "tracks.next":"tracks_next", "tracks.offset":"tracks_offset", "tracks.previous":"tracks_previous", "tracks.total":"tracks_total"})
                tracks = tracks.drop(columns=['track.album.artists', 'track.album.available_markets', 'track.album.images', 'track.artists', 'track.available_markets', 'images', 'tracks.items'])
                load_batch.append(tracks)
    except:
            pass
load_batch = pd.concat(load_batch)
load_batch.head()

100%|██████████| 25/25 [00:19<00:00,  1.26it/s]


,added_at,is_local,primary_color_x,added_by_external_urls_spotify,added_by_href,added_by_id,added_by_type,added_by_uri,track_album_album_type,track_album_external_urls_spotify,...,owner_href,owner_id,owner_type,owner_uri,tracks_href,tracks_limit,tracks_next,tracks_offset,tracks_previous,tracks_total
0,2023-02-06T09:33:41Z,False,None,https://open.spotify.com/user/rutin3rad,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,single,https://open.spotify.com/album/33ts6NqPgL6Pnmh...,...,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,https://api.spotify.com/v1/playlists/0ABVkZSG9...,100,None,0,None,100
1,2023-02-06T09:33:41Z,False,None,https://open.spotify.com/user/rutin3rad,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,single,https://open.spotify.com/album/3ZTGirX2U4761k9...,...,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,https://api.spotify.com/v1/playlists/0ABVkZSG9...,100,None,0,None,100
2,2023-02-06T09:33:41Z,False,None,https://open.spotify.com/user/rutin3rad,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,single,https://open.spotify.com/album/3LMjtMU3ltt5hZ1...,...,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,https://api.spotify.com/v1/playlists/0ABVkZSG9...,100,None,0,None,100
3,2023-02-06T09:33:41Z,False,None,https://open.spotify.com/user/rutin3rad,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,single,https://open.spotify.com/album/1YtQ91X08ecn4JE...,...,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,https://api.spotify.com/v1/playlists/0ABVkZSG9...,100,None,0,None,100
4,2023-02-06T09:33:41Z,False,None,https://open.spotify.com/user/rutin3rad,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,single,https://open.spotify.com/album/4pYBwEMBWE1FqUo...,...,https://api.spotify.com/v1/users/rutin3rad,rutin3rad,user,spotify:user:rutin3rad,https://api.spotify.com/v1/playlists/0ABVkZSG9...,100,None,0,None,100


## Upload Training Data to playlist_tracks Table

In [5]:
load_batch.to_sql('playlist_tracks', engine, if_exists='append')

545

## Identify All Artists in Every Song

In [6]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks ORDER BY track_uri''', engine)

outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)

df = pd.Series(anti_join['track_uri'])
base_list = []
for track in tqdm(df):
    try:
        df2 = pd.json_normalize(sp.track(track), record_path=['artists'])
        df2['track_uri'] = track
        df2 = df2[['name', 'track_uri']]
        base_list.append(df2)
    except:
        pass
df2 = pd.concat(base_list)
df2

  0%|          | 0/15605 [00:00<?, ?it/s]

## Get Row Numbers for Pivoting

In [ ]:
df2['RN'] = df2.groupby("track_uri")["name"].rank(method="first", ascending=True)
df2

,name,track_uri,RN
0,Avril Lavigne,spotify:track:00Mb3DuaIH1kjrwOku9CGU,1.0
0,2 Chainz,spotify:track:00QyLmjxaSEE8qIZQjBXBj,1.0
1,Wiz Khalifa,spotify:track:00QyLmjxaSEE8qIZQjBXBj,2.0
0,Ayo & Teo,spotify:track:01stPT7J3W6Zx45jj1f4nk,1.0
0,Internet Money,spotify:track:02kDW379Yfd5PzW5A6vuGt,3.0
...,...,...,...
0,SLANDER,spotify:track:7zKOcKHZYFiPKlJ4NdPl00,3.0
1,Said The Sky,spotify:track:7zKOcKHZYFiPKlJ4NdPl00,4.0
2,JT Roach,spotify:track:7zKOcKHZYFiPKlJ4NdPl00,2.0
3,Brondo,spotify:track:7zKOcKHZYFiPKlJ4NdPl00,1.0


## Pivot DataFrame to Fit all Artists in DataFrame

In [ ]:
df2 = df2.pivot(index='track_uri', columns=['RN'], values='name').reset_index()
df2 = df2.rename(columns={1.0:'artist1', 2.0:'artist2', 3.0:'artist3', 4.0:'artist4', 5.0:'artist5'})
df2 = df2[['track_uri', 'artist1', 'artist2', 'artist3', 'artist4', 'artist5']]
df2

RN,track_uri,artist1,artist2,artist3,artist4,artist5
0,spotify:track:00Mb3DuaIH1kjrwOku9CGU,Avril Lavigne,NaN,NaN,NaN,NaN
1,spotify:track:00QyLmjxaSEE8qIZQjBXBj,2 Chainz,Wiz Khalifa,NaN,NaN,NaN
2,spotify:track:01stPT7J3W6Zx45jj1f4nk,Ayo & Teo,NaN,NaN,NaN,NaN
3,spotify:track:02XnQdf7sipaKBBHixz3Zp,Lady Gaga,NaN,NaN,NaN,NaN
4,spotify:track:02kDW379Yfd5PzW5A6vuGt,Don Toliver,Gunna,Internet Money,NAV,NaN
...,...,...,...,...,...,...
664,spotify:track:7yPPKfvIn97GDkffy1oV7w,Akylla,Kill The Noise,NGHTMRE,SNAILS,NaN
665,spotify:track:7ycWLEP1GsNjVvcjawXz3z,A$AP Rocky,Skepta,NaN,NaN,NaN
666,spotify:track:7ytR5pFWmSjzHJIeQkgog4,DaBaby,Roddy Ricch,NaN,NaN,NaN
667,spotify:track:7zKOcKHZYFiPKlJ4NdPl00,Brondo,JT Roach,SLANDER,Said The Sky,NaN


## Upload All Artist Data to Database

In [ ]:
df2.to_sql('artists', con=engine, if_exists='append')

669

## Get All Audio Features

In [ ]:
df = pd.read_sql('''SELECT DISTINCT track_uri FROM playlist_tracks''', engine)
outer = df.merge(load_batch, how='outer', indicator=True)
anti_join = outer[(outer._merge=='left_only')].drop('_merge', axis=1)
df = pd.Series(anti_join['track_uri'])

base_list = []
for track in tqdm(df):
    try:
        df3 = pd.json_normalize(sp.audio_features(track))
        df3['track_uri'] = track
        df3 = df3[['track_uri', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature']]
        base_list.append(df3)
    except:
        pass
df3 = pd.concat(base_list)
df3

100%|██████████| 670/670 [01:02<00:00, 10.69it/s]


,track_uri,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,spotify:track:2JvzF1RMd7lE3KmFlsyZD8,0.837,0.364,8,-11.713,1,0.2760,0.14900,0.000000,0.2710,0.4630,123.984,213594,4
0,spotify:track:1Ah9ymSfMABC4ZKbV5e3Ha,0.832,0.791,4,-5.997,0,0.2880,0.14400,0.000000,0.0702,0.6020,95.007,270493,4
0,spotify:track:7LgNtm2LJiSWl2St8nhZNr,0.828,0.661,5,-4.505,0,0.2640,0.35100,0.000000,0.1370,0.3980,139.996,159425,4
0,spotify:track:0oTz1OfuL1uoGMNI3aLEkd,0.405,0.966,9,-2.063,1,0.1030,0.01440,0.000092,0.3090,0.0379,145.147,281199,4
0,spotify:track:6emaVMyM6xQkSF3zNqrndt,0.774,0.411,6,-7.412,0,0.0948,0.00397,0.000031,0.0722,0.2750,100.002,227880,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
0,spotify:track:25oOaleife6E2MIKmFkPvg,0.842,0.597,2,-6.336,1,0.0627,0.00252,0.000000,0.1240,0.2980,145.992,215307,4
0,spotify:track:1Ser4X0TKttOvo8bgdytTP,0.689,0.564,2,-6.226,1,0.1360,0.18700,0.000000,0.0721,0.1230,185.023,212947,4
0,spotify:track:7nc7mlSdWYeFom84zZ8Wr8,0.672,0.717,5,-7.476,1,0.2260,0.10300,0.000000,0.3980,0.4730,157.905,180380,4
0,spotify:track:5FEXPoPnzueFJQCPRIrC3c,0.871,0.440,11,-6.250,0,0.0761,0.49900,0.000004,0.1140,0.7990,137.996,221267,4


## Upload Audio Features to Database

In [ ]:
df3.to_sql('audio_features', con=engine, if_exists='append')

669

In [ ]:
all_data = pd.read_sql('''SELECT DISTINCT playlist_uri, playlist_name, owner_uri, owner_display_name, collaborative, description, followers_total
track_uri, track_name, artist1, artist2, artist3, artist4, artist5, album_uri, album_name, release_date, 
added_by_uri, added_at, track_album_total_tracks, af.duration_ms, "isExplicit", track_popularity, 
track_preview_url, video_thumbnail_url, danceability, energy, loudness, key, mode, acousticness, speechiness, 
instrumentalness, liveness, valence, tempo, time_signature
FROM playlist_tracks pt
JOIN artists a ON pt.track_uri = a.track_uri
JOIN audio_features af ON pt.track_uri = af.track_uri;''', engine)
f = open("s3.csv", "w")
f.truncate()
f.close()

all_data.to_csv('s3.csv', mode="w+")